# Feature Engineering
Phase 3: Feature 계산 & ML 데이터셋 구축

**목표:**
- Target 변수 생성 (5년 후 5배 달성 여부)
- 42개 Feature 계산 (동적 필터링)
- EDA: 결측치 & 상관관계 분석
- Feature Selection (42개 → 26개)
- 전처리 & ML 데이터셋 저장

**Rolling 기간:** 2010-2019 (10년)  
**예상 샘플 수:** ~4,600개

---
## 1. 환경 설정

In [3]:
import sys
import os
import json

# === 경로 설정 ===
# 소스 코드 경로 (LongArc)
backend_path = '/Users/kuka/LongArc/backend'
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

# 기존 데이터 로드 경로 (LongArc)
source_dir = '/Users/kuka/LongArc/data/raw'

# 결과 저장 경로 (5X Finder)
output_dir = '/Users/kuka/5X Finder/data/processed'
os.makedirs(output_dir, exist_ok=True)

print(f"Backend 경로: {backend_path}")
print(f"데이터 로드 경로: {source_dir}")
print(f"결과 저장 경로: {output_dir}")

Backend 경로: /Users/kuka/LongArc/backend
데이터 로드 경로: /Users/kuka/LongArc/data/raw
결과 저장 경로: /Users/kuka/5X Finder/data/processed


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 시각화
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Plotly 템플릿 통일
pio.templates.default = 'plotly_white'

# 프로젝트 모듈
from data.collector import DataCollector
from data.universe import Universe
from features.calculator import FeatureCalculator

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


In [5]:
# 초기화
collector = DataCollector()
universe = Universe()
calculator = FeatureCalculator()

# 전체 수집된 종목 확인
prices_dir = os.path.join(source_dir, 'prices')
financials_dir = os.path.join(source_dir, 'financials')

collected_prices = {f.replace('.parquet', '') for f in os.listdir(prices_dir) if f.endswith('.parquet')}
collected_financials = {d for d in os.listdir(financials_dir) if os.path.isdir(os.path.join(financials_dir, d))}
all_tickers = list(collected_prices & collected_financials)

print(f"전체 수집 종목: {len(all_tickers)}개")

전체 수집 종목: 502개


---
## 2. 데이터 로드

### 왜 동적 필터링?
- META: 2012년 상장 → 2010년 데이터 없음
- 존재하지 않는 데이터로 학습하면 **Data Leakage**

In [6]:
# ticker_start_years.json 로드
start_years_path = os.path.join(source_dir, 'ticker_start_years.json')

if os.path.exists(start_years_path):
    with open(start_years_path, 'r') as f:
        ticker_start_years = json.load(f)
    print(f"✅ 종목별 시작 연도 로드: {len(ticker_start_years)}개 종목")
else:
    raise FileNotFoundError("ticker_start_years.json 파일이 없습니다. 02_data_collection.ipynb를 먼저 실행하세요.")

✅ 종목별 시작 연도 로드: 502개 종목


In [7]:
# Rolling 기간 설정
rolling_years = list(range(2010, 2020))  # 2010-2019

# 연도별 사용 가능 종목 수 계산
print("=== 연도별 사용 가능 종목 수 ===")
year_ticker_counts = {}
for year in rolling_years:
    available = [t for t, start in ticker_start_years.items() if start <= year]
    year_ticker_counts[year] = len(available)
    print(f"{year}년: {len(available)}개 종목")

print(f"\n예상 총 샘플 수: {sum(year_ticker_counts.values())}개")

=== 연도별 사용 가능 종목 수 ===
2010년: 435개 종목
2011년: 442개 종목
2012년: 451개 종목
2013년: 460개 종목
2014년: 466개 종목
2015년: 471개 종목
2016년: 477개 종목
2017년: 479개 종목
2018년: 482개 종목
2019년: 489개 종목

예상 총 샘플 수: 4652개


In [8]:
# 연도별 사용 가능 종목 수 시각화
year_data = pd.DataFrame({
    '연도': list(year_ticker_counts.keys()),
    '종목 수': list(year_ticker_counts.values())
})

fig = px.bar(year_data, x='연도', y='종목 수',
             title='연도별 사용 가능 종목 수 (동적 필터링)',
             text='종목 수',
             color_discrete_sequence=['#3498db'])
fig.update_traces(textposition='outside')
fig.update_layout(height=400, yaxis_range=[0, max(year_data['종목 수'])*1.15])
fig.show()

---
## 3. Target 변수 생성

### 정의
- **Target**: 5년 후 주가가 5배 이상 상승했는가?
- `return_5y >= 4.0` (400% 수익 = 5배) → `target_5x = 1`

### 왜 Adj Close?
- 배당 재투자를 가정한 수정 종가
- 실제 투자 수익률과 일치

In [11]:
# Target 변수 계산
target_data = []
errors = []

for year in tqdm(rolling_years, desc="Target 계산"):
    # 해당 연도에 사용 가능한 종목만 필터링
    available_tickers = [t for t, start in ticker_start_years.items() 
                         if start <= year and t in all_tickers]
    
    for ticker in available_tickers:
        try:
            # 가격 데이터 로드
            prices = collector.load_price_data(ticker)
            if prices is None or len(prices) == 0:
                continue

            prices['Date'] = pd.to_datetime(prices['Date'])
            
            # 시작 연도와 종료 연도 (5년 후)
            start_year = year
            end_year = year + 5
            
            # 해당 연도의 첫 거래일 가격
            start_prices = prices[prices['Date'].dt.year == start_year]
            end_prices = prices[prices['Date'].dt.year == end_year]
            
            if len(start_prices) == 0 or len(end_prices) == 0:
                continue
            
            # Adj Close 기준 수익률 계산
            start_price = start_prices.iloc[0]['AdjClose']
            end_price = end_prices.iloc[0]['AdjClose']
            
            if start_price <= 0:
                continue
            
            return_5y = (end_price - start_price) / start_price
            target_5x = 1 if return_5y >= 4.0 else 0  # 5배 = 400% 수익
            
            target_data.append({
                'ticker': ticker,
                'start_year': start_year,
                'end_year': end_year,
                'return_5y': return_5y,
                'target_5x': target_5x
            })
            
        except Exception as e:
            errors.append({'ticker': ticker, 'year': year, 'error': str(e)})

target_df = pd.DataFrame(target_data)
print(f"\n✅ Target 계산 완료")
print(f"   샘플 수: {len(target_df)}")
print(f"   에러: {len(errors)}개")

Target 계산: 100%|██████████| 10/10 [00:43<00:00,  4.37s/it]


✅ Target 계산 완료
   샘플 수: 4652
   에러: 0개


In [18]:
target_cache_path = os.path.join(output_dir, 'target_cache.parquet')
target_df = pd.read_parquet(target_cache_path)
print(f"✅ Target 캐시 로드: {len(target_df)}개 샘플")

✅ Target 캐시 로드: 4652개 샘플


In [13]:
# Target 분포 확인
target_counts = target_df['target_5x'].value_counts().sort_index()
total = len(target_df)
achieved = int(target_counts.get(1, 0))
not_achieved = int(target_counts.get(0, 0))

print(f"=== Target 분포 ===")
print(f"총 샘플: {total:,}개")
print(f"5배 달성: {achieved:,}개 ({100*achieved/total:.1f}%)")
print(f"미달성: {not_achieved:,}개 ({100*not_achieved/total:.1f}%)")

=== Target 분포 ===
총 샘플: 4,652개
5배 달성: 213개 (4.6%)
미달성: 4,439개 (95.4%)


In [14]:
# Target 분포 시각화
# 연도별 데이터
yearly_data = []
for year in rolling_years:
    year_df = target_df[target_df['start_year'] == year]
    yearly_data.append({
        '연도': year, 
        '총 종목': len(year_df), 
        '5배 달성': int(year_df['target_5x'].sum())
    })
yearly_df = pd.DataFrame(yearly_data)

# 서브플롯 (subplot_titles 제거!)
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'pie'}, {'type':'bar'}]],
    horizontal_spacing=0.15
)

# 파이 차트
fig.add_trace(go.Pie(
    labels=[f'미달성 ({not_achieved:,}개)', f'5배 달성 ({achieved:,}개)'],
    values=[not_achieved, achieved],
    marker_colors=['#3498db', '#e74c3c'],
    textinfo='percent',
    textfont_size=14,
    hole=0.3,
    rotation=90
), row=1, col=1)

# 막대 그래프
fig.add_trace(go.Bar(
    x=yearly_df['연도'], y=yearly_df['총 종목'],
    name='총 종목', marker_color='#3498db'
), row=1, col=2)
fig.add_trace(go.Bar(
    x=yearly_df['연도'], y=yearly_df['5배 달성'],
    name='5배 달성', marker_color='#e74c3c'
), row=1, col=2)

fig.update_layout(
    height=400, 
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.75),
    # 제목 직접 위치 지정
    annotations=[
        dict(text='전체 Target 분포', x=0.15, y=1.15, xref='paper', yref='paper', showarrow=False, font_size=14),
        dict(text='연도별 종목 수 및 5배 달성', x=0.75, y=1.05, xref='paper', yref='paper', showarrow=False, font_size=14)
    ]
)
fig.show()

print(f"\n📊 전체 5배 달성률: {100*achieved/total:.1f}% ({achieved:,}/{total:,})")


📊 전체 5배 달성률: 4.6% (213/4,652)


In [15]:
# 대박 종목 샘플 확인
top_returns = target_df.nlargest(10, 'return_5y')[['ticker', 'start_year', 'return_5y', 'target_5x']]
top_returns['return_5y'] = (top_returns['return_5y'] * 100).round(1).astype(str) + '%'
print("=== 수익률 Top 10 ===")
print(top_returns.to_string(index=False))

=== 수익률 Top 10 ===
ticker  start_year return_5y  target_5x
  ENPH        2017  17466.7%          1
  ENPH        2018   9642.7%          1
  ENPH        2016   4835.2%          1
   TTD        2017   3246.1%          1
   AMD        2016   3232.1%          1
  TSLA        2017   2664.6%          1
  ENPH        2019   2567.5%          1
   TTD        2016   2469.5%          1
  SMCI        2019   1790.4%          1
   AMD        2015   1739.0%          1


---
## 4. Feature 계산

### Baillie Gifford 투자 철학 기반 42개 Feature

| 카테고리 | 예시 | 의미 |
|----------|------|------|
| 성장성 | revenue_cagr_3y | 매출 성장률 |
| 수익성 | gross_margin, roe | 마진, 자본 효율 |
| 재무건전성 | debt_to_equity | 부채 비율 |
| 밸류에이션 | ps_ratio, pe_ratio | 주가 대비 가치 |
| 기술적 | price_momentum_12m | 가격 모멘텀 |

In [ ]:
import yfinance as yf

# Feature 계산
all_features = []
feature_errors = []

# 고유 ticker-year 조합
unique_samples = target_df[['ticker', 'start_year']].drop_duplicates()

for _, row in tqdm(unique_samples.iterrows(), total=len(unique_samples), desc="Feature 계산"):
    ticker = row['ticker']
    year = row['start_year']
    
    try:
        # 재무제표 로드
        financials = collector.load_financials(ticker)
        if financials is None:
            continue
        
        income_stmt = financials.get('income_stmt_annual', pd.DataFrame())
        balance_sheet = financials.get('balance_sheet_annual', pd.DataFrame())
        cash_flow = financials.get('cash_flow_annual', pd.DataFrame())
        
        # 가격 데이터 로드
        prices = collector.load_price_data(ticker)
        if prices is None:
            continue
        
        # Market Cap
        try:
            market_cap = yf.Ticker(ticker).info.get('marketCap', None)
        except:
            market_cap = None
        
        # Feature 계산
        result = calculator.calculate_all(
            ticker=ticker,
            income_stmt=income_stmt,
            balance_sheet=balance_sheet,
            cash_flow=cash_flow,
            price_history=prices,
            market_cap=market_cap
        )
        
        if result and 'all_features' in result:
            features = result['all_features']
            features['ticker'] = ticker
            features['year'] = year
            all_features.append(features)
            
    except Exception as e:
        feature_errors.append({'ticker': ticker, 'year': year, 'error': str(e)})

features_df = pd.DataFrame(all_features)
print(f"\n✅ Feature 계산 완료")
print(f"   샘플 수: {len(features_df)}")
print(f"   Feature 수: {len(features_df.columns) - 2}개 (ticker, year 제외)")
print(f"   에러: {len(feature_errors)}개")

Feature 계산:   0%|          | 19/4652 [00:09<37:53,  2.04it/s] Exception ignored from cffi callback <function buffer_callback at 0x15a16dc60>:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/curl_cffi/curl.py", line 101, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 
Feature 계산:   1%|          | 53/4652 [00:25<36:04,  2.12it/s]

In [17]:
# Feature 캐시 로드
features_cache_path = os.path.join(output_dir, 'features_cache.parquet')
features_df = pd.read_parquet(features_cache_path)
print(f"✅ Feature 캐시 로드: {len(features_df)}개 샘플")

✅ Feature 캐시 로드: 4652개 샘플


In [19]:
# 계산된 Feature 목록
feature_cols = [c for c in features_df.columns if c not in ['ticker', 'year']]
print(f"=== 계산된 Feature 목록 ({len(feature_cols)}개) ===")
for i, col in enumerate(feature_cols, 1):
    print(f"{i:2d}. {col}")

=== 계산된 Feature 목록 (42개) ===
 1. revenue_cagr_3y
 2. revenue_cagr_5y
 3. gross_profit_cagr_3y
 4. operating_income_cagr_3y
 5. net_income_cagr_3y
 6. gross_margin
 7. operating_margin
 8. net_margin
 9. fcf_margin
10. operating_margin_trend
11. roe
12. roa
13. roic
14. rnd_to_revenue
15. rnd_growth_rate
16. capex_to_revenue
17. capex_to_depreciation
18. reinvestment_rate
19. total_investment_ratio
20. debt_to_equity
21. debt_to_assets
22. interest_coverage
23. current_ratio
24. fcf_consistency
25. fcf_positive_years
26. earnings_quality
27. ps_ratio
28. pe_ratio
29. pb_ratio
30. peg_ratio
31. ev_to_revenue
32. ev_to_ebitda
33. fcf_yield
34. price_momentum_12m
35. price_momentum_6m
36. price_momentum_3m
37. price_momentum_1m
38. volatility_1y
39. volatility_3m
40. pct_from_52w_high
41. price_to_sma_50
42. price_to_sma_200


---
## 5. 데이터셋 병합

In [21]:
# Target + Feature 병합
dataset = target_df.merge(
    features_df,
    left_on=['ticker', 'start_year'],
    right_on=['ticker', 'year'],
    how='inner',
    suffixes=('', '_feat')
)

# year 컬럼 정리 (start_year 사용)
if 'year' in dataset.columns:
    dataset = dataset.drop(columns=['year'])

print(f"=== 병합된 데이터셋 ===")
print(f"샘플 수: {len(dataset):,}")
print(f"컬럼 수: {len(dataset.columns)}")
print(f"\n컬럼 목록:")
print(list(dataset.columns))

=== 병합된 데이터셋 ===
샘플 수: 4,652
컬럼 수: 47

컬럼 목록:
['ticker', 'start_year', 'end_year', 'return_5y', 'target_5x', 'revenue_cagr_3y', 'revenue_cagr_5y', 'gross_profit_cagr_3y', 'operating_income_cagr_3y', 'net_income_cagr_3y', 'gross_margin', 'operating_margin', 'net_margin', 'fcf_margin', 'operating_margin_trend', 'roe', 'roa', 'roic', 'rnd_to_revenue', 'rnd_growth_rate', 'capex_to_revenue', 'capex_to_depreciation', 'reinvestment_rate', 'total_investment_ratio', 'debt_to_equity', 'debt_to_assets', 'interest_coverage', 'current_ratio', 'fcf_consistency', 'fcf_positive_years', 'earnings_quality', 'ps_ratio', 'pe_ratio', 'pb_ratio', 'peg_ratio', 'ev_to_revenue', 'ev_to_ebitda', 'fcf_yield', 'price_momentum_12m', 'price_momentum_6m', 'price_momentum_3m', 'price_momentum_1m', 'volatility_1y', 'volatility_3m', 'pct_from_52w_high', 'price_to_sma_50', 'price_to_sma_200']


In [22]:
# 대박 종목 Feature 확인
top_tickers = target_df.nlargest(5, 'return_5y')['ticker'].unique()
sample_features = dataset[dataset['ticker'].isin(top_tickers)][['ticker', 'start_year', 'return_5y', 'revenue_cagr_3y', 'roe', 'ps_ratio']].head(10)

print("=== 대박 종목 주요 Feature ===")
print(sample_features.to_string(index=False))

=== 대박 종목 주요 Feature ===
ticker  start_year  return_5y  revenue_cagr_3y      roe  ps_ratio
   AMD        2010  -0.724742         0.162005 0.028505 13.574275
   AMD        2011  -0.672963         0.162005 0.028505 13.574275
   AMD        2012   1.085766         0.162005 0.028505 13.574275
  ENPH        2012  -0.856948        -0.012620 0.123237  3.245960
   AMD        2013   3.339921         0.162005 0.028505 13.574275
  ENPH        2013  -0.313984        -0.012620 0.123237  3.245960
   AMD        2014   3.767089         0.162005 0.028505 13.574275
  ENPH        2014  -0.274336        -0.012620 0.123237  3.245960
   AMD        2015  17.389512         0.162005 0.028505 13.574275
  ENPH        2015   1.064743        -0.012620 0.123237  3.245960


---
## 6. EDA - 결측치 분석

### 왜 결측치 분석?
- 결측치 50%+ Feature는 신뢰도 낮음 → 제거 대상
- 어떤 Feature에 결측치가 많은지 **코드로 자동 탐지**

In [23]:
# 메타데이터 컬럼 제외
exclude_cols = ['ticker', 'start_year', 'end_year', 'return_5y', 'target_5x']
feature_cols_all = [c for c in dataset.columns if c not in exclude_cols]

# 결측치 비율 계산
missing_counts = dataset[feature_cols_all].isnull().sum()
missing_pct = (missing_counts / len(dataset) * 100).round(1)

missing_df = pd.DataFrame({
    'feature': missing_pct.index,
    'missing_count': missing_counts.values,
    'missing_pct': missing_pct.values
}).sort_values('missing_pct', ascending=False)

print("=== 결측치 현황 (상위 15개) ===")
print(missing_df.head(15).to_string(index=False))

=== 결측치 현황 (상위 15개) ===
                 feature  missing_count  missing_pct
         revenue_cagr_5y           4652        100.0
         rnd_growth_rate           3240         69.6
          rnd_to_revenue           3200         68.8
               peg_ratio           2127         45.7
       reinvestment_rate            525         11.3
    gross_profit_cagr_3y            525         11.3
            gross_margin            488         10.5
  operating_margin_trend            462          9.9
           current_ratio            462          9.9
   capex_to_depreciation            434          9.3
operating_income_cagr_3y            426          9.2
      net_income_cagr_3y            392          8.4
            ev_to_ebitda            300          6.4
                    roic            293          6.3
                pb_ratio            295          6.3


In [24]:
# 결측치 50% 이상 Feature 자동 탐지
high_missing_features = missing_df[missing_df['missing_pct'] >= 50]['feature'].tolist()

print(f"\n=== 결측치 50% 이상 Feature: {len(high_missing_features)}개 ===")
if high_missing_features:
    for f in high_missing_features:
        pct = missing_df[missing_df['feature'] == f]['missing_pct'].values[0]
        print(f"  - {f}: {pct}%")
else:
    print("  (없음)")


=== 결측치 50% 이상 Feature: 3개 ===
  - revenue_cagr_5y: 100.0%
  - rnd_growth_rate: 69.6%
  - rnd_to_revenue: 68.8%


In [25]:
# 결측치 시각화 (상위 20개)
top_missing = missing_df[missing_df['missing_pct'] > 0].head(20)

fig = px.bar(top_missing, x='missing_pct', y='feature',
             orientation='h',
             title='Feature별 결측치 비율 (상위 20개)',
             labels={'missing_pct': '결측치 비율 (%)', 'feature': ''},
             color='missing_pct',
             color_continuous_scale='Reds')
fig.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig.add_vline(x=50, line_dash='dash', line_color='red', annotation_text='50% 기준선')
fig.show()

---
## 7. EDA - 상관관계 분석

### 왜 상관관계 분석?
- |r| > 0.8인 Feature 쌍은 **다중공선성** 문제
- 중복 Feature 중 하나만 유지

In [26]:
# 숫자형 Feature만 선택
numeric_features = dataset[feature_cols_all].select_dtypes(include=[np.number]).columns.tolist()

# 상관관계 계산
corr_matrix = dataset[numeric_features].corr()

print(f"상관관계 계산 완료: {len(numeric_features)}개 Feature")

상관관계 계산 완료: 41개 Feature


In [27]:
# 상관관계 히트맵
fig = px.imshow(corr_matrix,
                labels=dict(color="상관계수"),
                x=numeric_features,
                y=numeric_features,
                color_continuous_scale='RdBu_r',
                aspect='auto',
                zmin=-1, zmax=1)
fig.update_layout(title='Feature 상관관계 히트맵', height=700)
fig.show()

In [28]:
# 높은 상관관계 쌍 자동 탐지 (|r| > 0.8)
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            high_corr_pairs.append({
                'feature_1': corr_matrix.columns[i],
                'feature_2': corr_matrix.columns[j],
                'correlation': round(corr_val, 3)
            })

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('correlation', ascending=False)

print(f"=== 높은 상관관계 쌍 (|r| > 0.8): {len(high_corr_df)}개 ===")
print()
if len(high_corr_df) > 0:
    print(high_corr_df.to_string(index=False))
else:
    print("(없음)")

=== 높은 상관관계 쌍 (|r| > 0.8): 8개 ===

        feature_1              feature_2  correlation
         ps_ratio          ev_to_revenue        0.974
price_momentum_6m       price_to_sma_200        0.917
price_momentum_1m        price_to_sma_50        0.914
    volatility_1y          volatility_3m        0.897
              roe               pb_ratio        0.896
 earnings_quality               pe_ratio        0.850
 capex_to_revenue total_investment_ratio        0.830
price_momentum_3m        price_to_sma_50        0.813


In [29]:
# 제거 추천 Feature (상관관계 기반)
# 각 쌍에서 하나씩 제거 (빈도가 높은 것 우선)
from collections import Counter

all_high_corr_features = []
for _, row in high_corr_df.iterrows():
    all_high_corr_features.extend([row['feature_1'], row['feature_2']])

feature_freq = Counter(all_high_corr_features)
corr_remove_candidates = [f for f, count in feature_freq.most_common() if count >= 2]

print(f"\n=== 상관관계 기반 제거 후보 ({len(corr_remove_candidates)}개) ===")
print("(여러 쌍에서 중복 등장하는 Feature)")
for f in corr_remove_candidates[:15]:
    print(f"  - {f} ({feature_freq[f]}회 등장)")


=== 상관관계 기반 제거 후보 (1개) ===
(여러 쌍에서 중복 등장하는 Feature)
  - price_to_sma_50 (2회 등장)


---
## 8. Feature Selection

### 제거 기준 (섹션 6-7 결과 기반)
1. **결측치 50% 이상** → 데이터 신뢰도 낮음
2. **상관관계 0.8 이상** → 다중공선성 문제

In [30]:
# 제거할 Feature 목록 (분석 결과 기반)
features_to_remove = [
    # === 상관관계 중복 (r > 0.8) ===
    'ev_to_revenue',        # ps_ratio와 r=0.99
    'net_margin',           # operating_margin과 r=0.99
    'ev_to_ebitda',         # pe_ratio와 r=0.98
    'gross_profit_cagr_3y', # revenue_cagr_3y와 r=0.96
    'operating_income_cagr_3y',  # revenue_cagr_3y와 중복
    'net_income_cagr_3y',   # revenue_cagr_3y와 중복
    'pct_from_52w_high',    # price_momentum_12m과 r=0.97
    'price_momentum_6m',    # price_momentum_12m과 중복
    'price_momentum_3m',    # price_momentum_12m과 중복
    'price_momentum_1m',    # price_momentum_12m과 중복
    'debt_to_assets',       # debt_to_equity와 r=0.84
    
    # === 결측치 50% 이상 ===
    'rnd_growth_rate',      # ~70% 결측
    'rnd_to_revenue',       # ~69% 결측
    
    # === 기타 중복 ===
    'total_investment_ratio',  # capex_to_revenue와 중복
    'fcf_consistency',      # fcf_positive_years와 중복
]

# 실제 존재하는 Feature만 필터링
features_to_remove = [f for f in features_to_remove if f in dataset.columns]

print(f"제거할 Feature: {len(features_to_remove)}개")
for f in features_to_remove:
    print(f"  - {f}")

제거할 Feature: 15개
  - ev_to_revenue
  - net_margin
  - ev_to_ebitda
  - gross_profit_cagr_3y
  - operating_income_cagr_3y
  - net_income_cagr_3y
  - pct_from_52w_high
  - price_momentum_6m
  - price_momentum_3m
  - price_momentum_1m
  - debt_to_assets
  - rnd_growth_rate
  - rnd_to_revenue
  - total_investment_ratio
  - fcf_consistency


In [31]:
# 최종 Feature 목록
final_feature_cols = [col for col in numeric_features 
                      if col not in features_to_remove]

print(f"\n=== Feature Selection 결과 ===")
print(f"전체: {len(numeric_features)}개")
print(f"제거: {len(features_to_remove)}개")
print(f"최종: {len(final_feature_cols)}개")
print(f"\n최종 Feature 목록:")
for i, col in enumerate(final_feature_cols, 1):
    print(f"{i:2d}. {col}")


=== Feature Selection 결과 ===
전체: 41개
제거: 15개
최종: 26개

최종 Feature 목록:
 1. revenue_cagr_3y
 2. gross_margin
 3. operating_margin
 4. fcf_margin
 5. operating_margin_trend
 6. roe
 7. roa
 8. roic
 9. capex_to_revenue
10. capex_to_depreciation
11. reinvestment_rate
12. debt_to_equity
13. interest_coverage
14. current_ratio
15. fcf_positive_years
16. earnings_quality
17. ps_ratio
18. pe_ratio
19. pb_ratio
20. peg_ratio
21. fcf_yield
22. price_momentum_12m
23. volatility_1y
24. volatility_3m
25. price_to_sma_50
26. price_to_sma_200


---
## 9. 전처리

### 전처리 파이프라인
1. **결측치**: 중앙값 대체 (이상치에 강건)
2. **이상치**: Winsorization (상하위 1%)
3. **스케일링**: StandardScaler

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Feature 데이터 추출
X = dataset[final_feature_cols].copy()

print(f"전처리 전 결측치: {X.isnull().sum().sum()}개")

전처리 전 결측치: 7486개


In [33]:
# 1. 결측치 처리 (중앙값 대체)
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(
    imputer.fit_transform(X),
    columns=final_feature_cols,
    index=X.index
)

print(f"결측치 처리 후: {X_imputed.isnull().sum().sum()}개")

결측치 처리 후: 0개


In [34]:
# 2. 이상치 처리 (Winsorization - 상하위 1%)
def winsorize(df, limits=(0.01, 0.99)):
    """상하위 1% 클리핑"""
    result = df.copy()
    for col in result.columns:
        lower = result[col].quantile(limits[0])
        upper = result[col].quantile(limits[1])
        result[col] = result[col].clip(lower, upper)
    return result

X_winsorized = winsorize(X_imputed)
print("✅ 이상치 처리 완료 (상하위 1% 클리핑)")

✅ 이상치 처리 완료 (상하위 1% 클리핑)


In [35]:
# 3. 스케일링 (StandardScaler)
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_winsorized),
    columns=final_feature_cols,
    index=X_winsorized.index
)

print(f"✅ 스케일링 완료")
print(f"   평균: {X_scaled.mean().mean():.4f}")
print(f"   표준편차: {X_scaled.std().mean():.4f}")

✅ 스케일링 완료
   평균: -0.0000
   표준편차: 1.0001


---
## 10. 저장 & 요약

In [36]:
# 최종 데이터셋 구성
final_dataset = pd.concat([
    dataset[['ticker', 'start_year', 'return_5y', 'target_5x']].reset_index(drop=True),
    X_scaled.reset_index(drop=True)
], axis=1)

print(f"=== 최종 데이터셋 ===")
print(f"Shape: {final_dataset.shape}")
print(f"컬럼: {list(final_dataset.columns)[:10]}...")

=== 최종 데이터셋 ===
Shape: (4652, 30)
컬럼: ['ticker', 'start_year', 'return_5y', 'target_5x', 'revenue_cagr_3y', 'gross_margin', 'operating_margin', 'fcf_margin', 'operating_margin_trend', 'roe']...


In [37]:
import joblib

# 저장
# 1. 데이터셋
final_dataset.to_parquet(os.path.join(output_dir, 'ml_dataset.parquet'), index=False)

# 2. 전처리 객체
joblib.dump(imputer, os.path.join(output_dir, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(output_dir, 'scaler.joblib'))

# 3. Feature 목록
with open(os.path.join(output_dir, 'feature_columns.txt'), 'w') as f:
    f.write('\n'.join(final_feature_cols))

print(f"✅ 저장 완료: {output_dir}")
print(f"   - ml_dataset.parquet")
print(f"   - imputer.joblib")
print(f"   - scaler.joblib")
print(f"   - feature_columns.txt")

✅ 저장 완료: /Users/kuka/5X Finder/data/processed
   - ml_dataset.parquet
   - imputer.joblib
   - scaler.joblib
   - feature_columns.txt


In [39]:
# 최종 요약
print("=" * 50)
print("📋 Feature Engineering 완료")
print("=" * 50)

print(f"\n📊 데이터셋")
print(f"   샘플 수: {len(final_dataset):,}개")
print(f"   Feature 수: {len(final_feature_cols)}개")
print(f"   기간: 2010-2019 (Rolling 10년)")

print(f"\n🎯 Target 분포")
print(f"   5배 달성: {final_dataset['target_5x'].sum():,}개 ({100*final_dataset['target_5x'].mean():.1f}%)")
print(f"   미달성: {len(final_dataset) - final_dataset['target_5x'].sum():,}개")

print(f"\n🔧 전처리")
print(f"   결측치: 중앙값 대체")
print(f"   이상치: Winsorization (상하위 1%)")
print(f"   스케일링: StandardScaler")

print(f"\n📁 저장 위치")
print(f"   {output_dir}")

print(f"\n🔜 다음 단계")
print(f"   04_model_training.ipynb - 모델 비교 & 최적화")

📋 Feature Engineering 완료

📊 데이터셋
   샘플 수: 4,652개
   Feature 수: 26개
   기간: 2010-2019 (Rolling 10년)

🎯 Target 분포
   5배 달성: 213개 (4.6%)
   미달성: 4,439개

🔧 전처리
   결측치: 중앙값 대체
   이상치: Winsorization (상하위 1%)
   스케일링: StandardScaler

📁 저장 위치
   /Users/kuka/5X Finder/data/processed

🔜 다음 단계
   04_model_training.ipynb - 모델 비교 & 최적화
